In [ ]:
#Load libs

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000 
from PIL import Image, ImageOps
import fcsparser
import sys

#set up working dir

directory = str("/mnt/SERVER-CRCT-STORAGE/CRCT_Imagin/CORDELIER Pierre/HaloData/21-003.IMMCORE.C2v1/Halo archive 2021-12-13 16-38 - v3.3.2541/ObjectData/")
directory = str("/mnt/SERVER-CRCT-STORAGE/CRCT_Imagin/CORDELIER Pierre/HaloData/21-003.IMMCORE.C1v1/Halo archive 2021-12-13 13-32 - v3.3.2541/ObjectData")
outpath = '/home/mouneem/tysserand/CORDELIER_PIERRE/'
log = '/home/mouneem/tysserand/CORDELIER_PIERRE/'+'log.txt'


In [ ]:
# FCS TO CSV
# iterate over files in
# that directory
i = 0
failed = []
for filename in os.listdir(directory):
    try:    
        i += 1
        f = os.path.join(directory, filename)
        # checking if it is a file
        filename_csv = filename
        if os.path.isfile(f) :
            #meta, data = fcsparser.parse(f, reformat_meta=True)
            data = pd.read_csv(f) 
            #df1 = df[['a', 'b']] // df1 = df.iloc[:, 0:2] 
            image_name = data.iat[0,0].split("\\")[-1].split("/")[-1]
            level = data.iat[0,1]
            outname = ".".join([ image_name,level,'csv' ])
            data["x"] = (data["XMax"] + data["XMin"] ) / 2
            data["y"] = (data["YMax"] + data["YMin"] ) / 2

            col_is_marker = []
            for col in list(data.columns):
                if col.count('+') + col.count('-') > 1:
                    col_is_marker.append(True)
                else:
                    col_is_marker.append(False)


            marker = data.loc[:, col_is_marker]
            # C1_markers = marker['PanCK+CD3-CD8-CD20-'].astype(bool) 
            # C2_markers = marker['PanCK-CD3+CD8+'].astype(bool) 
            # C3_markers = marker['PanCK-CD3+CD8-'].astype(bool) 
            # C4_markers = marker['PanCK-CD3-CD20+'].astype(bool) 
            # C5_markers = marker['PanCK-CD3+CD20+'].astype(bool) 

            # classes = ['Other']*data.shape[0]

            # for i in range(len(classes)):
            #     if C1_markers.values[i]:
            #         classes[i] = 'C1'
            #     elif C2_markers.values[i]:
            #         classes[i] = 'C2'
            #     elif C3_markers.values[i]:
            #         classes[i] = 'C3'
            #     elif C4_markers.values[i]:
            #         classes[i] = 'C4'
            #     elif C5_markers.values[i]:
            #         classes[i] = 'C5'


            M1_markers = marker['CD206+CD163+CD68-CD16-'].astype(bool) | marker['CD206+CD163+CD68-CD16+'].astype(bool)
            M2a_markers = marker['CD206+CD163+CD68-CD16-'].astype(bool) | marker['CD206+CD163+CD68-CD16+'].astype(bool) 
            M2c_markers = marker['CD206-CD163+CD68-CD16+'].astype(bool) 

            classes = ['Other']*data.shape[0]

            for i in range(len(classes)):
                if M1_markers.values[i]:
                    classes[i] = 'M1'
                elif M2c_markers.values[i]:
                    classes[i] = 'M2c'
                elif M2a_markers.values[i]:
                    classes[i] = 'M2a'

            from collections import Counter
            print(filename, Counter(classes))

            data["class"] = classes

            nodes = data[['x','y',"class"]]
            nodes.to_csv(outpath + "csv_levels/" + outname,  mode='a', header = False)


            # ratio = 10  CD206-CD163+CD68-CD16-
            # nodes = nodes.sample(n = round(len(nodes) / ratio) )
            # coords = nodes.loc[:,['x','y']].values

            # plt.scatter(coords[:,0] , coords[:,1] ,   zorder=10, s = 1)
        
            # plt.savefig(outpath+filename_csv+'.png')
            
            # plt.cla()

    except PermissionError:
        failed.append(filename)
open(log , 'a+').write("\n".join(failed))